# Introdução à Inteligência Artificial - PEE / 1 - Formulação

# Guião Laboratorial
(20/Setembro:24/Setembro-2021)

## Introdução

Vamos ver como poderemos, usando a linguagem Python, resolver problemas formulados de acordo com o paradigma do espaço de estados (PEE). Para isso, há que completar dois passos:
1. Definir o problema;
2. Utilizar um algoritmo de procura para o resolver.

Nesta aula vamos tratar apenas da formulação, i.e., da definição do problema.

Recordando, para formularmos um problema de acordo com esta metododologia, precisamos de:
* **Estados**: Idealizar uma representação para o que vamos considerar um estado. Notem que o estado deve ser mínimo, apenas deve conter a informação que muda com as acções; 
* **Estado Inicial**: Identificar o estado inicial;
* **Teste de Objectivo**: Verificar se um estado satisfaz o objectivo, sendo assim, um dos estados considerados finais;
* **Acções**: Para cada estado, caracterizar rigorosamente as acções de mudança de estado, de que modo incrementam os custos dos caminhos, e quais os estados resultantes.

## Módulo *searchPlus.py* - breve explicação

Este módulo é uma variante muito ligeira do search.py que está disponível no repositório [aima-python](https://github.com/aimacode/aima-python), que contém a implementação (em Python) da generalidade dos algoritmos descritos no livro da disciplina (Russell & Norvig). 
Muitas das definições deste módulo não serão utilizadas. Vamos apenas concentrar-nos em algumas das suas classes e funções.
No essencial, é disponiblizado o seguinte:
* A classe **Problem**, que vamos utilizar para definir os problemas;
* A classe **Node**, que representa um ***nó de procura***, utilizada pelos algoritmos de procura implementados;
* A implementação de vários algoritmos de procura.

Para esta aula dedicada à formulação, apenas precisamos de usar a classe **Problem** do módulo, que tem de ser importado, mas não deverá alterá-lo.

In [8]:
from searchPlus import *

## Exemplo base 
Para já, vamos ilustrar com um exemplo de problema que consiste num grafo de estados, com um estado inicial I e final F. Notem que há dois arcos bidireccionais, entre I e A e entre I e B. O problema consiste em encontrar uma sequência de acções (um plano) que nos leva de I a F.


<img src="Figuras\figura-ee-T-1.png" alt="Drawing" style="width: 300px;"/>

### Representação de um estado
Não há qualquer restrição quanto ao tipo que o estado pode assumir (pode ser um inteiro, uma string, uma lista, um tuplo, etc, ou um tipo definido recorrendo a uma nova classe).

Neste exemplo do grafo abstracto, o estado é identificável por uma letra.
Em Python, vamos considerar que um estado é uma string de um caracter que corresponde aos nós do grafo na figura. Vamos ter um espaço de estados muito reduzido, formado pelo seguinte conjunto de estados: {'I', 'A','B', 'C', 'D', 'F'}. 

### A classe **Problem**
Esta classe funciona como uma classe *abstracta*. Para definir um problema concreto é necessário criar uma sua sub-classe. A classe **Problem** tem um construtor em que se passa o estado inicial e o estado final ou conjunto de estados finais e a função de teste do objectivo **goal_test** apenas verifica se um estado é igual ao estado final ou se pertence ao conjunto dos estados finais, respectivamente; tem também um método **path_cost**, por defeito, que considera que todas as acções possuem custo 1, o que é o caso do nosso grafo em cima.

In [9]:
## Apenas a definição da classe, ainda sem qualquer conteúdo
class ProblemaGrafo(Problem) :
    pass 

Um problema é criado fornecendo a identificação do estado inicial e, opcionalmente, algo que representa o objectivo. Note que o objectivo pode ser um estado uma lista de estados ou qualquer outra informação diferente de um estado mas que represente o objectivo desejado.

Veja-se a assinatura do construtor da classe **Problem**, a qual por omissão tem o objectivo a None:

```python
def __init__(self, initial, goal=None):
    """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal
```
Por exemplo, para criarmos o problema de encontrar um caminho entre **'I'** e **'F'** no espaço acima, poderíamos fazer:

In [25]:
problema_1 = ProblemaGrafo('I','F')

Neste caso, os atributos **initial** e **goal** guardam explicitamente os estados inicial e final

In [26]:
print(problema_1.initial)
print(problema_1.goal)

I
F


#### Definição do objectivo
Há duas formas de definir o teste de satisfação do objectivo:
1. Indicando explicitamente o seu valor ou os seus valores possíveis, utilizando o parâmetro ***goal*** do construtor; o parâmetro ***goal*** pode ser também uma lista de estados, nos casos em que existam vários estados finais.
Quando é esse o caso, não se precisa de redefinir o método **goal_test** da classe Problem, que apresentamos a seguir:


```python
def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return state in self.goal
        else:
            return state == self.goal

```
Este método retornará ```True``` nos casos em que o estado fornecido (**state**) seja o estado final ou membro dos estados finais.

2. Redefinindo o método **goal_test** na classe que define o nosso problema, método esse que codifica as condições de satisfação do estado final, sem a necessidade de conhecer explicitamente o estado final ou a lista deles.

No exemplo acima, utilizámos a primeira opção, definindo como objectivo o estado **F**, precisamente porque sabemos o estado que queremos atingir: 'F'.

#### A função ***__eq__***
Quando existem vários estados finais que podemos enumerar, faz sentido usar o método ***goal_test*** que verifica se um estado é o estado final ou membro de uma lista de estados finais, usando o operador **in**. No , nosso exemplo, como os estados são strings, não é preciso redefinir o teste de igualdade. Mas, convém realçar que se os estados fossem instâncias de uma classe e não strings, como dois objectos diferentes mas com o mesmo conteúdo não são naturalmente iguais, teríamos de redefinir o método **__eq__**. Veremos um exemplo em que é necessário redefinir o **__eq__** mais abaixo.

#### Operadores
A implementação dos operadores de mudança de estado é feita com base na definição de dois métodos:
* ```actions(self, state)``` - Este método, dado um estado, devolve a lista de todas as acções possíveis nesse estado; a representação concreta do que é uma acção fica em aberto.
* ```result(self, state, action)``` - Dados um estado e uma acção, este método devolve o estado resultante da execução da acção ***action***, no estado ***state***.

A definição destes dois métodos na classe que define o problema é ***obrigatória***. Notem que a classe **Problem** tem os métodos **actions** e **result** por implementar, devolvendo erro quando não são sobrepostos pela subclasse de **Problem**. 

```python
    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError
```

como se verifica a seguir, ao dar erro.

In [24]:
problema_1.actions(problema_1.initial)

NotImplementedError: 

Continuando o mesmo exemplo, antes de definir estas funções, temos que representar o grafo que define o espaço de estados ilustrado acima. Iremos guardá-lo num atributo estático da classe.

Em python, as variáveis estáticas apenas existem ao nível da classe e nunca são instanciadas. Se mudarmos o valor de uma variável estática numa instância de uma classe, a mudança afectará o seu valor em todas as outras instâncias. Do mesmo modo, os métodos estáticos não se referem a nenhuma instância da classe e podem ser chamados fora da classe.

Uma possiblidade é utilizar um dicionário para representar o grafo e que guardaremos então numa variável estática da classe a que chamamos de grafo:
```python
grafo = {'I':['A','B'],
         'A':['C','D','I'],
         'B':['D','F','I'],
         'C':[],
         'D':['C','F'],
         'F':[]}
```
Nesta representação de um grafo, a chave é um estado, sendo o valor correspondente a lista dos possíveis sucessores. 

Como os custos dos arcos do grafo são homogéneos, não é necessário representá-los.
A classe **Problem** atribui o custo 1 a cada acção por defeito, através do método **path_cost**. Mas note que o método recebe como parâmetro o custo acumulado do estado inicial até state1 (c) e adiciona-lhe 1, que é o custo de ir de state1 para state2. 

```python
def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1
   
 ```
 
 Não precisaremos de redefinir este método mas apenas o método **actions** e o método **result**. Vamos a isso.

##### Definindo o método **actions**
Neste exemplo abstracto, uma acção é simplesmente uma transição entre dois estados, X e Y, que poderemos representar pela string **"Ir de X para Y"**.

Assim, podemos ter a seguinte definição do método **actions**:
```python
def actions(self,estado) :
    """A acção é uma string com o formato seguinte:
        ir de x para y, em que x e y são estados
    """
    sucessores = self.grafo[estado]  # obter lista dos sucessores
    accoes = map(lambda x : "ir de {} para {}".format(estado,x),sucessores) # compor as strings que representam cada uma das possíveis acções
    return list(accoes)
```

Notem que a função **map** aplica uma função a todos os elementos de uma lista. Neste caso, a função é uma função lambda ou anónima, criada e usada na hora.

Atenção que poderíamos escolher outra forma de representar as acções, em tuplos, por exemplo, em vez de ser em strings. (x,y) poderia significar ir de x para y. ('I','A) representaria ir do estado I para o estado A. 

##### Definindo o método **result**
O método **result** recebe como um dos parâmetros uma acção, que terá que ter o formato determinado pelo método **actions**. Não tem de se preocupar com as pré-condições das acções porque é ***sempre*** o método **actions** que trata disso.

Como cada acção é uma cadeia de caracteres em que o último é o estado seguinte ou sucessor, destino da acção, o que queremos é obter a última palavra de uma *acção*. Vamos usar o método **split** das strings para partir a frase numa lista de palavras e depois vamos buscar a última, usando o índice -1 de uma lista.
```python
def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        Com o split dividimos a string numa lista de palavras e 
        depois sacamos a última que é o estado resultante da acção
        """
        return accao.split()[-1]
```

Vejamos então como fica a definição completa da classe:

In [13]:
class ProblemaGrafo(Problem) :
    grafo = {'I':['A','B'],
             'A':['C','D','I'],
             'B':['D','F','I'],
             'C':[],
             'D':['C','F'],
             'F':[]}

    def actions(self,estado) :
        """A acção é uma string com o formato seguinte:
        ir de x para y, em que x e y são estados
        """
        sucessores = self.grafo[estado]
        accoes = map(lambda x : "ir de {} para {}".format(estado,x),sucessores)
        return list(accoes)
        #
        # alternativamente:
        # accoes = ["ir de {} para {}".format(estado,x) for x in sucessores]
        # return accoes
        # 
        # alternaivamente:
        # accoes = list()
        # for x in sucessores :
        #     accoes.append("ir de {} para {}".format(estado,x))
        # return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        Com o split dividimos a string numa lista de palavras e 
        depois sacamos a última que é o estado resultante da acção
        """
        return accao.split()[-1]

Podemos então redefinir o nosso problema e verificar se os métodos dão o resultado esperado:

In [14]:
p1 = ProblemaGrafo('I','F')

Vamos listar todos os estados do espaço de estados, que correspondem às chaves do dicionário, guardado no atributo chamado grafo.

In [15]:
print("Estados:")
for estado in p1.grafo.keys():
    print(estado)

Estados:
I
A
B
C
D
F


Listemos todas as acções possíveis a partir do estado inicial do problema

In [16]:
p1.actions(p1.initial)

['ir de I para A', 'ir de I para B']

Vejamos qual o resultado de executar a acção 'ir de I para B' a partir de I

In [17]:
p1.result('I','ir de I para B')

'B'

Testemos se 'B' é o estado final

In [18]:
p1.goal_test('B')

False

Vamos aplicar a primeira accão ao estado inicial e verifiquemos onde estamos

In [19]:
p1.result(p1.initial,p1.actions(p1.initial)[0])

'A'

Vamos testar a satisfação do objectivo e calcular o custo progressivamente à medida que vamos executando a primeira acção dado o estado inicial e a segunda dado o estado resultante, começando com um custo de 0.

In [20]:
custo=0
e0 = p1.initial
print('Em',e0,'com custo',custo)
print('Cheguei ao objectivo?',p1.goal_test(e0))
a1 = p1.actions(e0)[0]
e1 = p1.result(e0,a1)
custo = p1.path_cost(custo,e0,a1,e1)
print('Após executar a 1ª acção:', a1, 'passei para ',e1, ', com custo',custo)
print('Cheguei ao objectivo?',p1.goal_test(e1))
a2 = p1.actions(e1)[0]
e2 = p1.result(e1,a2)
custo = p1.path_cost(custo,e1,a2,e2)
print('Após executar a 2ª acção:', a2, ', passei para ',e2, ', com custo',custo)
print('Cheguei ao objectivo?',p1.goal_test(e1))

Em I com custo 0
Cheguei ao objectivo? False
Após executar a 1ª acção: ir de I para A passei para  A , com custo 1
Cheguei ao objectivo? False
Após executar a 2ª acção: ir de A para C , passei para  C , com custo 2
Cheguei ao objectivo? False


## Exercício 1
Crie um problema da classe **ProblemaGrafo** com o estado inicial 'A' e estados finais 'F' e 'E'.

In [23]:
Prob = ProblemaGrafo('A',['F','E'])# invocação do construtor aqui...

## Exercício 2
Como poderá mudar o grafo da classe **ProblemaGrafo** de modo a poderem utilizar diferentes grafos e não apenas este? Ficam aqui duas sugestões e podem implementar as duas: (1) mudem o valor do atributo estático antes ou depois de criarem uma instância da classe; (2) refaçam a classe de modo a que o grafo seja passado como parâmetro no construtor, sendo um atributo dinâmico e privado de cada objecto do tipo **ProblemaGrafo**.

In [27]:
def redefineGraf(Grafo):
    ProblemaGrafo.grafo = Grafo # sugestão 1

In [29]:
# sugestão 2
class ProblemaGrafo_Mine(Problem) :
        
    grafo = {'I':['A','B'],
             'A':['C','D','I'],
             'B':['D','F','I'],
             'C':[],
             'D':['C','F'],
             'F':[]}
    
    def __init__(self,Graf = grafo):
        self.grafo = Graf

    def actions(self,estado) :
        """A acção é uma string com o formato seguinte:
        ir de x para y, em que x e y são estados
        """
        sucessores = self.grafo[estado]
        accoes = map(lambda x : "ir de {} para {}".format(estado,x),sucessores)
        return list(accoes)
        #
        # alternativamente:
        # accoes = ["ir de {} para {}".format(estado,x) for x in sucessores]
        # return accoes
        # 
        # alternaivamente:
        # accoes = list()
        # for x in sucessores :
        #     accoes.append("ir de {} para {}".format(estado,x))
        # return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        Com o split dividimos a string numa lista de palavras e 
        depois sacamos a última que é o estado resultante da acção
        """
        return accao.split()[-1]

## Grafo abstracto com custos


Olhemos agora para um grafo em que os custos não são homogéneos. Por exemplo este:

<img src="Figuras\figura-ee-T-2.png" alt="Drawing" style="width: 300px;"/>

Vamos representar este grafo com custos num dicionário em que cada chave é um estado e em que o valor é um novo dicionário, sendo a chave o estado sucessor e o valor o custo do arco.
O grafo será guardado no atributo grafo da nova classe

``` python
grafo = {'I':{'A':2,'B':5},
             'A':{'C':2,'D':4,'I':2},
             'B':{'D':1,'F':5,'I':5},
             'C':{},
             'D':{'C':3,'F':2},
             'F':{}}
```

Para além disso, precisamos de adaptar o método **actions** para lidar com o novo dicionário 

```python
def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes
```

e de criar um método **path_cost** que se vai sobrepor ao método da super-classe **Problem**. Lembram-se que este último devolve sempre 1 como custo de qualquer arco.
Este método recebe o custo actual do caminho (c) desde o estado inicial até ao estado ***state1***, calculado pelo algoritmo de procura. A acção (***action***) provoca uma transição entre ***state1*** e ***state2*** e o custo da acção é adicionado a ***c***.
Para sabermos qual o custo da acção que leva do ***state1*** ao ***state2*** teremos de ir ler no grafo:

``` python
def path_cost(self, c, state1, action, state2):
        return c + self.grafo[state1][state2]
```

Vejamos então a definição completa da nova classe:

In [30]:
class ProblemaGrafoCustos(Problem) :
    grafo = {'I':{'A':2,'B':5},
             'A':{'C':2,'D':4,'I':2},
             'B':{'D':1,'F':5,'I':5},
             'C':{},
             'D':{'C':3,'F':2},
             'F':{}}

    def __init__(self,initial = 'I', final = 'F') :
        super().__init__(initial,final)
        
    def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        """
        return accao.split()[-1]
    
    def path_cost(self, c, state1, action, state2):
        """Assume-se que action é da forma 'ir de <state1> para <state2>'
        """
        return c + self.grafo[state1][state2]


Vamos então criar uma instância de ***ProblemaGrafoCustos*** e executar algumas operações.

In [31]:
p = ProblemaGrafoCustos()
custo = 0
print("Estado inicial:",p.initial, 'com custo',custo)
print("Estado final:",p.goal)
actions = p.actions(p.initial)
print('Cheguei ao objectivo?',p.goal_test(e0))
print("As acções do estado inicial:",str(actions))
e1 = p.result(p.initial,actions[1])
print("Partindo de", p.initial,"executo a acção de " + actions[1]+" e vou parar em :",e1)
custo = p.path_cost(0,p.initial,actions[1],e1)
print("O custo desde o início é:",custo)
print('Cheguei ao objectivo?',p.goal_test(e1))
e2 = p.result(e1,p1.actions(e1)[1])
print("Partindo de", e1,"executo a acção de " + p1.actions(e1)[1]+" e vou parar em :",e2)
custo = p.path_cost(custo,e1,actions[1],e2)
print("O custo desde o início é:",custo)
print('Cheguei ao objectivo?',p1.goal_test(e2))

Estado inicial: I com custo 0
Estado final: F
Cheguei ao objectivo? False
As acções do estado inicial: ['ir de I para A', 'ir de I para B']
Partindo de I executo a acção de ir de I para B e vou parar em : B
O custo desde o início é: 5
Cheguei ao objectivo? False
Partindo de B executo a acção de ir de B para F e vou parar em : F
O custo desde o início é: 10
Cheguei ao objectivo? True


## O Problema da inversão das setas
<img src="Figuras\6arrows.PNG" alt="Drawing" style="width: 300px;"/>
Vamos ver como poderemos, usando a linguagem Python, formular o problema da inversão das setas

Recordando o problema: Imagine que temos seis setas dispostas de cima para baixo, em que as primeiras 3 estão orientadas para a esquerda e as 3 últimas para a direita e que queremos obter seis setas com orientação alternadas, quando o único movimento possível é inverter a orientação de duas setas adjacentes. Queremos conhecer a sequência de acções que levam de um estado inicial a um estado final.

Se partirmos da situação inicial e aplicarmos a inversão das duas primeiras setas, obteremos todas orientadas para a direita, excepto a terceira.

### Representação dos estados
Poderíamos representar o estado por uma estrutura de dados simples, como uma lista. No entanto, para uma modelização mais object-oriented, e para puxar pelo vosso Python, vamos representar os estados deste problema como sendo objectos com um atributo que é uma lista de comprimento 6 em que da esquerda para a direita representamos a orientação das setas, ordenadas  de cima para baixo, em que "d" indica que uma seta aponta para a direita e "e" representa o sentido da seta para a esquerda. A situação inicial é dada pela lista: [e,e,e,d,d,d]

Vamos ter um atributo ***setas*** onde guardamos a lista.

Só representamos numa classe porque queremos associar ao atributo um conjunto de métodos. Assim, precisamos de 3 métodos, pelo menos 

    i) que inverta uma seta: de "e" para "d" e vice-versa.

    ii) que inverta um par de setas adjacentes indicando o primeiro índice: 1 para inverter a primeira e a segunda, ..., 5 para inverter a quinta e sexta setas.

    iii) que imprima as setas de um modo mais ou menos "pretty".  

In [32]:
class EstadoSetas :

    """Um estado do problema da inversao das setas
        Uma lista de 6 setas (e's ou d's), indicando para cada seta se está orientada
        para a esquerda para para a direita
        A ordem da esquerda para a direita corresponde às setas de cima para baixo
    """
    def __init__(self,setas = ["e","e","e","d","d","d"]) :
        """ Construtor: por omissão um estado corresponde ao estado inicial do problema"""
        self.setas = setas

        
    def flip(self,seta) :
        """ Inversão do sentido de uma seta: de e para d e de d para e"""
        if seta=="e":
            return "d"
        else:
            return "e"

    def inverte(self,n) :
        """ Inverte duas setas, na posição n e n+1. A primeira seta está na posição 0
            e para inverter a primeira e a segunda, de cima para baixo, implica n = 0.
            Gera uma nova lista.
        """
        copye = []
        for i in range(len(self.setas)) :
            if i == n-1 or i == n :
                copye.append(self.flip(self.setas[i]))
            else:
                copye.append(self.setas[i])
        return EstadoSetas(copye)
        
    def __str__(self) :
        """ Cria uma string com a orientação das setas separadas pelo símbolos -.
            Esta é a string devolvida pelo método print"""
        return str("-".join(self.setas))

Podemos criar uma instância de um estado, por exemplo, o caso inicial do problema,  desta maneira, sem passar quaisquer argumentos:

In [33]:
x = EstadoSetas()
print(x)

e-e-e-d-d-d


Se quisermos criar uma nova instância, com outra combinação de orientações teremos de "pisar" o estado inicial por omissão.

In [34]:
x = EstadoSetas(["d","e","d","d","d","e"])
print(x)

d-e-d-d-d-e


Se quisermos inverter a segunda e terceira seta de x, faremos:

In [35]:
print("Começamos com:",x)
y = x.inverte(1)
print('Dado',x,"inverte a primeira e a segunda setas:",y)
z = y.inverte(2)
print('Dado',y,"inverte a segunda e terceira setas:",z)

Começamos com: d-e-d-d-d-e
Dado d-e-d-d-d-e inverte a primeira e a segunda setas: e-d-d-d-d-e
Dado e-d-d-d-d-e inverte a segunda e terceira setas: e-e-e-d-d-e


Podemos agora inverter de novo a 1ª e 2ª setas , depois de inverter a 2ª e 3ª, regressando à configuração inicial.

In [36]:
f = z.inverte(2).inverte(1)
print('Agora façamos o inverso, invertamos a 2ª e a 3ª primeiro e depois a 1ª e a 2ª',f)

Agora façamos o inverso, invertamos a 2ª e a 3ª primeiro e depois a 1ª e a 2ª d-e-d-d-d-e


Testemos e são iguais, a original e a que resulta das quatro inversões em espelho.

In [37]:
print("Iguais?",f == x)
print("f =",f)
print("x =",x)

Iguais? False
f = d-e-d-d-d-e
x = d-e-d-d-d-e


Como pudemos ver, os dois estados não são considerados iguais embora os seus atributos *setas* sejam iguais. São duas instâncias diferentes. Têm de ser iguais se os respectivos atributos 'setas' forem iguais.
Assim, a função de teste ***goal_test()*** herdada de **Problem** não irá funcionar porque para verificar se um objecto é membro de uma lista é preciso que esse objecto seja membro da lista, i.e., igual a algum elemento da lista.

In [38]:
EstadoSetas(['d','e','d','e','d','e']) in [EstadoSetas(['d','e','d','e','d','e']),EstadoSetas(['e','d','e','d','e','d'])]

False

In [39]:
EstadoSetas(['d','e','d','e','d','e']) == EstadoSetas(['d','e','d','e','d','e'])

False

Vamos definir o método mágico **\_\_eq\_\_** de modo a que os estados sejam considerados iguais quando as ***setas*** forem iguais.

``` python
    def __eq__(self,estado) :
        """Definir em que circunstância os dois estados são considerados iguais.
        Necessário para os algoritmos de procura em grafo.
        """
        return self.setas == estado.setas
```



Vejemos a redefinição da classe **Setas()**

In [40]:
class EstadoSetas :

    """Um estado do problema da inversao das setas
        Uma lista de 6 setas (e's ou d's), indicando para cada seta se está orientada
        para a esquerda para para a direita
        A ordem da esquerda para a direita corresponde às setas de cima para baixo
    """
    def __init__(self,setas = ["e","e","e","d","d","d"]) :
        """ Construtor: por omissão um estado corresponde ao estado inicial do problema"""
        self.setas = setas

        
    def flip(self,seta) :
        """ Inversão do sentido de uma seta: de e para d e de d para e"""
        if seta=="e":
            return "d"
        else:
            return "e"

    def inverte(self,n) :
        """ Inverte duas setas, na posição n e n+1. A primeira seta está na posição 0
            e para inverter a primeira e a segunda, de cima para baixo, implica n = 0.
            Gera uma nova lista.
        """
        copye = []
        for i in range(len(self.setas)) :
            if i == n-1 or i == n :
                copye.append(self.flip(self.setas[i]))
            else:
                copye.append(self.setas[i])
        return EstadoSetas(copye)
    
    def __eq__(self,estado) :
        """Definir em que circunstância os dois estados são considerados iguais.
        Necessário para os algoritmos de procura em grafo.
        """
        return self.setas == estado.setas
        
    def __str__(self) :
        """ Cria uma string com a orientação das setas separadas pelo símbolos -.
            Esta é a string devolvida pelo método print"""
        return str("-".join(self.setas))
    

Voltemos a repetir as acções que fizemos lá trás.

In [41]:
x = EstadoSetas(["d","e","d","d","d","e"])
print("Começamos com:",x)
y = x.inverte(1)
print("Agora, inverte a primeira e a segunda:",y)
z = y.inverte(2)
print("E inverte a segunda e terceira setas:",z)
f = z.inverte(2).inverte(1)
print("Façamos o inverso das duas inversões:",f)
print("Os dois estados (o original e este que resulta de 4 acções) serão iguais?",f==x)

Começamos com: d-e-d-d-d-e
Agora, inverte a primeira e a segunda: e-d-d-d-d-e
E inverte a segunda e terceira setas: e-e-e-d-d-e
Façamos o inverso das duas inversões: d-e-d-d-d-e
Os dois estados (o original e este que resulta de 4 acções) serão iguais? True


Confirmemos mais uma vez que dois objectos diferentes com setas iguais, são iguais

In [42]:
EstadoSetas(["d","e","d","d","d","e"]) == EstadoSetas(["d","e","d","d","d","e"])

True

### Definindo a classe ProblemaSetas, subclasse de Problem
Notem que não é preciso **goal_test** nem **path_cost**. Não precisamos do **goal_test** porque apenas queremos verificar se um estado está na lista de estados finais que são conhecidos, o que o **goal_test** herdado de **Problem** já faz. Não precisamos do **path_cost** porque cada acção custa sempre 1, que é o caso do **path_cost** herdado de **Problem**.

In [43]:
from searchPlus import *


class ProblemaSetas(Problem) :
    
    def __init__(self,initial = EstadoSetas(["e","e","e","d","d","d"]),goal=[EstadoSetas(["e","d","e","d","e","d"]),EstadoSetas(["d","e","d","e","d","e"])]) :
        """ Basta invocar o construtor de Problem com os dois argumentos initial e goal."""
        super().__init__(initial,goal)

    
    def actions(self,estado) :
        """ 5 acções: 1, 2, 3, 4 e 5.
            A acção 1 corresponde a inverter as setas de índices 0 e 1 da lista
            A acção 5 coorresponde a inverter as setas de índices 4 e 5, as últimas duas """
        accoes = [1,2,3,4,5]
        
        return accoes 

    def result(self,estado,acao) :
        """ Basta-nos invocar o método inverte definido na classe EstadoSetas. 
            No caso de a acção não ser reconhecida imprime mensagem de erro e devolve None"""
        if acao in self.actions(estado) :
          resultante = estado.inverte(acao)
        else :
            raise "Há aqui qualquer coisa mal>> acao não reconhecida"
 
        return resultante

Criemos então uma instância da classe **ProblemaSetas** e imprimamos o seu estado inicial, o definido por omissão.

In [44]:
p = ProblemaSetas()
print(p.initial)

e-e-e-d-d-d


In [45]:
x = EstadoSetas(['d','e','d','e','d','e'])
print(x,"satisfaz o objectivo?",p.goal_test(x))
y = EstadoSetas(['d','d','d','e','d','e'])
print(y,"satisfaz o objectivo?",p.goal_test(y))

d-e-d-e-d-e satisfaz o objectivo? True
d-d-d-e-d-e satisfaz o objectivo? False


Quais as acções aplicáveis ao estado inicial?

In [46]:
p.actions(p.initial)

[1, 2, 3, 4, 5]

Apliquemos a acção 1 (inversão da 1ª e 2ª setas) ao estado inicial

In [47]:
s1 = p.result(p.initial,1)
print(s1)

d-d-e-d-d-d


Confirmemos que se voltarmos a aplicar a mesma acção regressamos ao estado inicial

In [48]:
s2 = p.result(s1,1)
s2 == p.initial

True

Confirmemos que o ***goal_test()*** herdado está a funcionar bem. 
Vamos aplicar as acções 1, 2, 4 e 5 sobre o estado inicial e verifiquemos se o estado resultante é final. 

In [49]:
custo=0
print(p.initial)
print('Custo total:',custo)
print('Inverte setas 1 e 2')
s1 = p.result(p.initial,1)
print(s1)
custo=p.path_cost(custo,p.initial,1,s1)
print('Custo total:',custo)
print('Inverte setas 2 e 3')
s2 = p.result(s1,2)
print(s2)
custo=p.path_cost(custo,s1,2,s2)
print('Custo total:',custo)
print('Inverte setas 3 e 4')
s3 = p.result(s2,4)
print(s3)
custo=p.path_cost(custo,s1,4,s2)
print('Custo total:',custo)
print('Inverte setas 4 e 5')
s4 = p.result(s3,5)
print(s4)
custo=p.path_cost(custo,s3,1,s4)
print('Custo total:',custo)
print("Final?",p.goal_test(s4))

e-e-e-d-d-d
Custo total: 0
Inverte setas 1 e 2
d-d-e-d-d-d
Custo total: 1
Inverte setas 2 e 3
d-e-d-d-d-d
Custo total: 2
Inverte setas 3 e 4
d-e-d-e-e-d
Custo total: 3
Inverte setas 4 e 5
d-e-d-e-d-e
Custo total: 4
Final? True


## Exercício 3
Adapta o problema da inversão das 6 setas para qualquer número de setas.

## Exercício 4
Supõe que tens um número arbitrário de moedas de 50, 20, 10, 5, 2 e 1 cêntimos, e que pretendes dar o troco no valor de N cêntimos, utilizando o menor número de moedas. Formula o problema em Python, seguindo o paradigma do Espaço de Estados e a plataforma aimas-python, de modo a poder resolver o problema de saber quais as moedas a utilizar para formar qualquer troco desejado.
<img src="Figuras\centimos.PNG" alt="Drawing" style="width: 200px;"/>

## Exercício 5
Resolve o problema da ordenação das panquecas. Dada uma pilha de panquecas de vários tamanhos, podes ordená-las de modo descrescente, com a maior em baixo e a mais pequena no topo? Possuis uma espátula com a qual podes inverter as i panquecas do topo, sendo i maior do que 1 e menor ou igual ao número de panquecas. Podes inverter as duas de topo, ou as 3 de topo, ..., ou todas. A figura em baixo ilustra o problema com uma espátula com i=3; no topo a espátula agarra 3 panquecas e no fundo aparecem invertidas:

<img src="https://upload.wikimedia.org/wikipedia/commons/0/0f/Pancake_sort_operation.png" alt="Drawing" style="width: 200px;"/>


## Exercício 6
Formula o problema das rãs e sapos saltitantes.
Neste puzzle, temos uma linha formada por quadrados, com N quadrados à esquerda preenchidos com rãs azuis e N quadrados à direita preenchidos com sapos vermelhos, e um quadrado livre no meio. Os sapos só podem saltar para a esquerda e as rãs para a direita. As rãs podem deslocar-se para a direita para uma casa livre ou saltar sobre uma rã ou sapo, para uma casa livre à sua direita. Os sapos movem-se de forma análoga, mas para a esquerda. O objectivo é inverter a configuração inicial, tendo os sapos à esquerda, a casa livre e as rãs todas à direita. Na figura seguinte, N é igual a 2.

![](https://upload.wikimedia.org/wikipedia/commons/2/2f/ToadsAndFrogs.png)

## Exercício 7
Considera que que queremos saltar para o passado e realizar uma viagem na linha de metro de Lisboa da época (ver figura). O viajante pretende planear uma viagem entre duas estações que lhe minimize o número de transbordos.

Formula este problema de acordo com o paradigma do espaço de estados, usando a classe **Problem** incluída no [aima-python](https://github.com/aimacode/aima-python).
Podes representar uma sub-parte da linha de metro, incluindo todas as estações de interface e pelo menos 3 estações de cada linha que não sejam de interface entre linhas, escolhidas a teu belo prazer. Nota que a modelização deve estar adaptada de modo a poderes aplicar a qualquer linha de metro, seja a de Lisboa no passado ou no presente, a de Londres, Berlim ou São Petersburgo.
<img src="Figuras\LinhaMetroLisboar.PNG" alt="Drawing" style="width: 500px;"/>

## Exercício 8
Considera o problema dos quadrados iluminados.
É dada uma grelha quadrada, 3x3, na qual alguns quadrados estão iluminados (claros) e outros na escuridão (mais escuros), como por exemplo na grelha da esquerda, acima. O objectivo é atingir, no menor número de passos possível, um estado em que todos os quadrados estejam iluminados (grelha à direita).  
De modo a atingir esse objectivo, pode-se escolher um qualquer quadrado e inverter o seu estado (na figura acima, foi escolhido o quadrado C3). Só que essa acção tem também o efeito de inverter os estados de todos os quadrados adjacentes (na horizontal e vertical). Cada quadrado contém um pequeno diagrama mostrando quais os outros quadrados que mudam quando esse é invertido.  
No exemplo acima, ao ser escolhido o quadrado C3, vão também inverter o estado os quadrados B3 e C2, tal como ilustrado. Assim, C3 passou de escuro a iluminado; B3 passou de iluminado a escuro; e C2 passou de escuro a iluminado.
![figura](Figuras\grelha.png)

Formula este problema de acordo com o paradigma do espaço de estados, e não te esqueças de ter um método para imprimir o tabuleiro, facilitando a sua visualização.